# 주문횟수, unique점포이용 거리계산 주문추천

## 1. 필요한 패키지 불러오기 Pandas, Numpy 등

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, euclidean

## 2. 데이터 불러오기

In [3]:
df1 = pd.read_csv("./12월주문자료.CSV",encoding="cp949",engine="python",sep=";")
df1["주문횟수"]=np.repeat(1,len(df1))
df1["FC명"]=  [x.replace("<","").replace(">","") for x in df1.FC명]
only = df1[df1.위탁구분명!="범용"]
every = df1[df1.위탁구분명=="범용"]
df11 = df1.pivot_table(index="매출처명",columns="SKU코드",values="주문횟수",aggfunc=np.sum)
df22 = pd.DataFrame(df11.sum())
df22.columns = ["orders"]
unique점포1 = []
for i2 in df11.columns:
    tt1 = df11[i2].dropna()
    u1 = len(tt1)
    unique점포1 += [u1]
df22["unique shop"]=unique점포1

In [4]:
cosin1 = pairwise_distances(np.array(df22),metric = 'cosine')
cosin_df = pd.DataFrame(cosin1)
cosin_df.columns = df22.index
cosin_df.index = df22.index

In [5]:
df1["분류코드"]= df1["대분류코드"].astype(str)+df1["중분류코드"].astype(str)+df1["소분류코드"].astype(str)

In [6]:
df1["소분류코드"][df1["소분류코드"].notnull()]=df1["소분류코드"][df1["소분류코드"].notnull()].astype(np.int).astype(str)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
df1.columns

Index(['주문일자', 'FC 코드', 'FC명', '매출처코드', '매출처명', '위탁구분', '위탁구분명', '대분류코드',
       '대분류명', '중분류코드', '중분류명', '소분류코드', '소분류명', 'SKU코드', 'SKU명', '주문수량(수불단위)',
       '수불단위', '주문횟수', '분류코드'],
      dtype='object')

In [8]:
df2=df1.loc[:,["주문일자","FC 코드","SKU코드","매출처코드","위탁구분","분류코드"]]

In [9]:
df2.columns = ["JUMU_DATE","FC_CODE","SKU_CODE", "STORE_ID","ONLYJUST","CLA_CODE"]

In [10]:
df1["분류코드"]=str(df1.대분류코드)+str(df1.중분류코드)+str(df1.소분류코드)

Unknown metric eucliden. Valid metrics are ['euclidean', 'l2', 'l1', 'manhattan', 'cityblock', 'braycurtis', 'canberra', 'chebyshev', 'correlation', 'cosine', 'dice', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule', 'wminkowski'], or 'precomputed', or a callable

In [11]:
def rec(x,fc):
    df2 = only[only["FC 코드"]==fc].append(every)
    df5 = df2.loc[:,["SKU명","SKU코드","소분류명","위탁구분명"]]
    df5 = df5.drop_duplicates()
    new_df = pd.DataFrame({"a":cosin_df.index,"b":cosin_df.iloc[:,0]})
    new_df.columns = ["SKU코드","distance"]
    new_df2 = df5.merge(new_df,on = "SKU코드", how = "left")
    cat1 = new_df2.소분류명[new_df2["SKU코드"]==x].unique()[0]
    new_df2 = new_df2[new_df2["SKU코드"]!=x]
    cats = new_df2[new_df2.소분류명==cat1].sort_values("distance",ascending=True)
    catd = new_df2[new_df2.소분류명!=cat1].sort_values("distance",ascending=True)
    df1.SKU명[df1["SKU코드"]==x], df1.FC명[df1["FC 코드"]==fc]
    cats.index.name=df1.SKU명[df1["SKU코드"]==x].unique()[0]+", "+df1.FC명[df1["FC 코드"]==fc].unique()[0]
    catd.index.name=df1.SKU명[df1["SKU코드"]==x].unique()[0]+", "+df1.FC명[df1["FC 코드"]==fc].unique()[0]
    cats.columns = ["SKUname","SKUcode","cate","use","Distance"]
    catd.columns = ["SKUname","SKUcode","cate","use","Distance"]
    return(cats,catd)

In [12]:
s,d = rec(106468,10175)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'SKU코드' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  import sys


In [13]:
s

,SKUname,SKUcode,cate,use,Distance
"상하목장 유기농 딸기우유 125ml*24, 스페인클럽",,,,,
3108,매일연유_20kg/box,112953,가공우유,범용,0.012237
3317,멸균우유#_1L*12ea(폴란드),117886,가공우유,범용,0.013606
353,상하목장 유기농 바나나우유 125ml*24,106469,가공우유,범용,0.024962
180,바이오 플레인 테이스티_호텔&레스토랑(매일)1000g*16ea,114682,가공우유,범용,0.028538
815,크리밀연유(서강)_500g*20ea/Box,119880,가공우유,범용,0.035151
2680,바나나는 원래 하얗다(매일유업)_235ml*24ea/box,123063,가공우유,범용,0.046769
836,에어로탑휘핑크림(리치코리아)_1L*12ea/box,110438,가공우유,범용,0.055566
1838,허쉬쵸코드링크(매일)235ML*24,105712,가공우유,범용,0.056361
1387,차이티_946ml*12ea/box,122717,가공우유,범용,0.067418


In [14]:
d

,SKUname,SKUcode,cate,use,Distance
"상하목장 유기농 딸기우유 125ml*24, 스페인클럽",,,,,
276,할라피뇨_슬라이스(델솔)3KG/EA,100008,소스/드레싱류,범용,0.000000e+00
2772,매콤 오돌뼈볶음_300g*20ea/box,119129,일반냉동식품,범용,0.000000e+00
2540,바닐라시럽(다빈치)750ml*12ea,101933,시럽,범용,1.110223e-16
1783,건빵(신흥)6.5KG*pk,107088,가공식품기타,범용,1.110223e-16
2431,무지_케익박스_3호_27*27*19_50ea/box,109070,종이용기,범용,1.110223e-16
599,BNK프리미엄 미니와플_20G*20EA*4PK,122544,일반빵,범용,1.110223e-16
956,냉동가리비관자_1kg*10pk/box,120971,냉동패류,범용,1.110223e-16
1602,딸기쨈(디스펜팩) 오뚜기 12GR*480EA=BOX,107602,편의식품,범용,1.110223e-16
1112,다진마늘_800g*12ea/Box,119702,냉동채소,범용,1.107212e-06
